In [1]:
# string
from langchain_ollama import ChatOllama

llm = ChatOllama(model="llama3.2:1b")

# llm.invoke("What is the capital of the France?")

In [2]:
# prompt template
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt_template = PromptTemplate(
    template="What is the capital of {country}? Return the name of the capital only.",
    input_variables=["country"],
)

prompt = prompt_template.invoke({"country": "France"})
print(prompt)

ai_message = llm.invoke(prompt_template.invoke({"country": "France"}))
print(ai_message)

output_parser = StrOutputParser()

answer = output_parser.invoke(llm.invoke(prompt_template.invoke({"country": "France"})))

text='What is the capital of France? Return the name of the capital only.'
content='Paris.' additional_kwargs={} response_metadata={'model': 'llama3.2:1b', 'created_at': '2025-03-13T08:01:21.392357Z', 'done': True, 'done_reason': 'stop', 'total_duration': 8132660916, 'load_duration': 7744735041, 'prompt_eval_count': 40, 'prompt_eval_duration': 371000000, 'eval_count': 3, 'eval_duration': 15000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)} id='run-8bb1a5d0-5ae6-4fbb-8efe-effc968fdb3d-0' usage_metadata={'input_tokens': 40, 'output_tokens': 3, 'total_tokens': 43}


In [3]:
# 모든 llm이 BaseChatModel을 상속받음(ollama, gpt, claude, etc)
# BaseChatModel은 runnable을 상속받음
# runnable에 invoke가 있음. 그렇기 때문에 다 Invoke를 하는 것임 
# 그렇기 때문에 langchain에서는 모든 것이 runnable임 

# runnable들로 chain을 만들 것임
# runnable은 실행가능한 것들
# |는 파이프를 의미함

# output_parser.invoke(llm.invoke(prompt_template.invoke({"country": "France"}))을 반대로 넘겨주면 됨
# 앞에서 뒤로 넘겨주다보니 가독성이 좋음 
capital_chain = prompt_template | llm | output_parser
capital_chain.invoke({"country": "France"}) # capital_chain도 runnable이다보니 invoke가 됨. 또한 파이프에 연결될 수 있음 

'Paris'

In [7]:
country_prompt = PromptTemplate( 
    template="""Guess the name of the country in the country based on the following information:
    {information}
    Return the name of the country only.
    """,
    input_variables=["information"],
)

country_chain = country_prompt | llm | output_parser
country_chain.invoke({"information": "This country is very famous for its wine in Europe."})

'Italy.'

In [8]:
final_chain = country_chain | capital_chain

# country_chain의 결과를 capital_chain에게 줌.
# 근데 capital_chain은 country가 필요함

In [9]:
final_chain = {"country":country_chain} | capital_chain

In [10]:
# 그리고 final_chain의 Invoke를 다음과 같이 함
final_chain.invoke({"information":"This country is very famous for its wine in Europe"})

'Rome'

In [11]:
# RunnablePassthrough라는게 있음
from langchain_core.runnables import RunnablePassthrough
final_chain = {"information": RunnablePassthrough()} | {"country": country_chain} | capital_chain

In [12]:
final_chain.invoke({"information":"This country is very famous for its wine in Europe"}) #{}가 하나만 필요하기때문에 굳이 information: 이렇게 안써도 됨

# invoke안의 정보가 바로 위 Cell의 Information으로 들어가서 위의 country_chain의 invoke에 있는 information에 들어감
# 그럼 그 답변이 country가 되서, 또 그 country가 capital_chain에게 들어감 

'Rome'

In [ ]:
# RunnablePassthrough의 또다른 팁
# {}가 2개면, (information, continent)
# key를 다 지정해줘야함
# final_chain.invoke({"information": "This country is very famous for its wine in Europe", "continent": "Europe"})

# 아래처럼하면 에러남
# final_chain.invoke({"This country is very famous for its wine in Europe"})

In [14]:
country_prompt = PromptTemplate( 
    template="""Guess the name of the country in the {continent} based on the following information:
    {information}
    Return the name of the country only.
    """,
    input_variables=["information",'continent'],
)

country_chain = country_prompt | llm | output_parser

In [15]:
# Runnable Passthrough
from langchain_core.runnables import RunnablePassthrough
final_chain = {"information": RunnablePassthrough(), "continent": RunnablePassthrough()} | {"country": country_chain} | capital_chain

In [18]:
final_chain.invoke({"information": "This country is very famous for its wine in Europe", "continent": "Europe"})

'Rome'